In [1]:
import pandas as pd

In [29]:
df_sweet = pd.read_csv('./mordred/sweet_descriptor.csv')
df_bitter = pd.read_csv('./mordred/bitter_descriptor.csv')

/var/folders/v6/r0fxlspx31j27x6g02zq7kmc0000gn/T/ipykernel_17234/2905487981.py:1: DtypeWarning: Columns (92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,138,139,140,141,147,148,149,150,165,166,167,168,174,175,176,177,183,184,185,186,191,192,193,194,195,200,201,202,203,204,209,210,211,212,213,219,220,221,222,228,229,230,231,236,237,238,239,240,241,242,243,244,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,344,345,346,347,348,354,355,356,357,363,364,365,366,381,382,383,384,390,391,392,393,399,400,401,402,407,408,409,410,411,416,417,418,419,420,425,426,427,428,429,435,436,437,438,444,445,446,447,452,453,454,455,460,461,462,463,468,469,470,471,484,485,486,487,492,493,494,495,500,501,502,503,508,509,510,511,516,517,518,519,524,525,526,527,532,533,534,535,540,541,542,543,548,549,550,551,556,557,558,559,564,565,566,567,580,581,582,583,588,589,590,591,596,597,598,599,604,605,606,607

In [51]:
df_sweet_features = df_sweet[df_sweet.columns.difference(['Name', 'SMILES','Canonical SMILES'])]
df_bitter_features = df_bitter[df_bitter.columns.difference(['Name', 'SMILES','Canonical SMILES'])]

df_sweet_features = df_sweet_features.drop('Target', axis=1)
df_bitter_features = df_bitter_features.drop('Target', axis=1)

In [52]:
df_sweet_features.apply(lambda x: pd.to_numeric(x, errors='coerce') )
mask_sweet = df_sweet_features.applymap(lambda x: isinstance(x, (int, float)))
df_sweet_features = df_sweet_features.where(mask_sweet)

df_bitter_features.apply(lambda x: pd.to_numeric(x, errors='coerce') )
mask_bitter = df_bitter_features.applymap(lambda x: isinstance(x, (int, float)))
df_bitter_features = df_bitter_features.where(mask_bitter)

In [53]:
df_sweet_set = pd.concat([df_sweet[['Name', 'SMILES','Canonical SMILES']], df_sweet_features], axis=1)
df_bitter_set = pd.concat([df_bitter[['Name', 'SMILES','Canonical SMILES']], df_bitter_features], axis=1)

In [54]:
from sklearn.model_selection import train_test_split
X_train_sweet, X_test_sweet, y_train_sweet, y_test_sweet = train_test_split(df_sweet_features, df_sweet[['Target']], test_size=0.2, random_state=42)
X_train_bitter, X_test_bitter, y_train_bitter, y_test_bitter = train_test_split(df_bitter_features, df_bitter[['Target']], test_size=0.2, random_state=42)


In [84]:
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
# sweet_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
# bitter_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
sweet_imputer = KNNImputer(n_neighbors=8, weights="uniform")
bitter_imputer = KNNImputer(n_neighbors=8, weights="uniform")

In [85]:
X_train_sweet_tf = sweet_imputer.fit_transform(X_train_sweet)
X_test_sweet_tf = sweet_imputer.transform(X_test_sweet)

X_train_bitter_tf = bitter_imputer.fit_transform(X_train_bitter)
X_test_bitter_tf = bitter_imputer.transform(X_test_bitter)

## Model Training

In [86]:
from sklearn.metrics import accuracy_score, precision_score, average_precision_score

In [87]:
def evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Accuracy:",accuracy_score(y_test,y_pred))
    print("Precision:",precision_score(y_test,y_pred))
    print("Average Precision:",average_precision_score(y_test,y_pred))

In [88]:
from xgboost import XGBClassifier

clf_xgboost = XGBClassifier()

In [89]:
from catboost import CatBoostClassifier

clf_catboost = CatBoostClassifier(
    iterations=100, 
    learning_rate=0.2, 
    logging_level='Silent'
)

In [90]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(
    n_estimators=250
)

In [91]:
print("Evaluating XGBoost")
evaluate(clf_xgboost, X_train_sweet_tf, y_train_sweet, X_test_sweet_tf, y_test_sweet)

print("Evaluating CatBoost")
evaluate(clf_catboost, X_train_sweet_tf, y_train_sweet, X_test_sweet_tf, y_test_sweet)

print("Evaluating RandomForest")
evaluate(clf_rf, X_train_sweet_tf, y_train_sweet, X_test_sweet_tf, y_test_sweet)

Evaluating XGBoost
Accuracy: 0.8888888888888888
Precision: 0.8875968992248062
Average Precision: 0.855731370266254
Evaluating CatBoost
Accuracy: 0.8846153846153846
Precision: 0.8778625954198473
Average Precision: 0.8482323444155506
Evaluating RandomForest


/var/folders/v6/r0fxlspx31j27x6g02zq7kmc0000gn/T/ipykernel_17234/2116390059.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


Accuracy: 0.8867521367521367
Precision: 0.8932806324110671
Average Precision: 0.8566723131940523
